In [ ]:
# Import necessary libraries
import os
import pandas as pd
import numpy as np
from PIL import Image
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
import torch
import timm

# Define paths to test images and CSV file containing test image IDs
TEST_PATH = './data/test'         
TEST_CSV = './data/test_ids.csv'  


In [ ]:
# Mapping of soil type names to integer labels and vice versa
label2id = {"Alluvial soil": 0, "Black Soil": 1, "Clay soil": 2, "Red soil": 3}
id2label = {v: k for k, v in label2id.items()}  # Reverse mapping for predictions


In [ ]:
# Custom Dataset class for loading soil images for inference/testing
class SoilDataset(Dataset):
    def __init__(self, df, image_dir, transform=None, is_test=False):
        self.df = df                   # DataFrame containing image info
        self.image_dir = image_dir     # Directory where images are stored
        self.transform = transform     # Image transformations (resize, normalize etc.)
        self.is_test = is_test         # Flag for test mode (no labels)

    def __len__(self):
        return len(self.df)            # Number of samples in the dataset

    def __getitem__(self, idx):
        row = self.df.iloc[idx]                                  # Get one row from DataFrame
        img_path = os.path.join(self.image_dir, row['image_id']) # Full path to the image
        image = Image.open(img_path).convert("RGB")             # Open image and convert to RGB
        
        if self.transform:
            image = self.transform(image)                        # Apply transforms if provided
        
        if self.is_test:
            return image, row['image_id']                        # For test: return image and its id
        else:
            label = label2id[row['soil_type']]                   # For training: return image and label
            return image, label


In [ ]:
# Define transforms to apply on images during inference/validation
val_transform = transforms.Compose([
    transforms.Resize((224, 224)),              # Resize images to 224x224 (model input size)
    transforms.ToTensor(),                       # Convert PIL Image to tensor
    transforms.Normalize([0.5]*3, [0.5]*3)      # Normalize with mean=0.5 and std=0.5 for all 3 channels
])


In [ ]:
def predict_test(models, test_df):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # Use GPU if available

    # Create dataset and dataloader for test images
    test_dataset = SoilDataset(test_df, TEST_PATH, transform=val_transform, is_test=True)
    test_loader = DataLoader(test_dataset, batch_size=32)

    all_probs = []  # To store prediction probabilities from each model (fold)

    for model_path in models:
        # Load model architecture without pretrained weights (weights loaded manually)
        model = timm.create_model('convnext_base', pretrained=False, num_classes=4)
        model.load_state_dict(torch.load(model_path, map_location=device))  # Load trained weights
        model.to(device)
        model.eval()  # Set model to evaluation mode

        fold_probs = []  # To store probabilities for this fold

        with torch.no_grad():
            for images, _ in test_loader:  # Images and image_ids, ignore ids here
                images = images.to(device)
                outputs = model(images)                          # Forward pass
                probs = torch.softmax(outputs, dim=1).cpu().numpy()  # Convert logits to probabilities
                fold_probs.append(probs)

        all_probs.append(np.vstack(fold_probs))  # Stack all batch probs for this fold

    # Average probabilities across all folds (ensembling)
    avg_probs = np.mean(all_probs, axis=0)
    final_preds = np.argmax(avg_probs, axis=1)  # Choose class with max average probability
    
    return final_preds


In [ ]:
if __name__ == '__main__':
    # Load test CSV which contains the image IDs to predict on
    test_ids_df = pd.read_csv(TEST_CSV)

    # List of saved model checkpoints from training (3-fold ensemble)
    model_paths = [
        "best_model_3fold_fold1.pth",
        "best_model_3fold_fold2.pth",
        "best_model_3fold_fold3.pth"
    ]

    print("Running test inference...")
    preds = predict_test(model_paths, test_ids_df)  # Run inference and get predictions

    # Prepare submission DataFrame: add predicted soil types to test IDs
    submission_df = test_ids_df.copy()
    submission_df['soil_type'] = [id2label[p] for p in preds]  # Map predicted class indices back to soil names

    # Save submission CSV file
    submission_df.to_csv("submission.csv", index=False)
    print("Submission saved to submission.csv")
